In [1]:
import pandas as pd

# Import external modules for data processing

from filters import CompartmentFilter, CASNumberFilter, ExcludeCompartmentFilter, apply_filter, normalize_cas
from data_fetching import fetch_ar6_ghg_data, fetch_hodnebrog_data
from data_enrichment import enrich_ar6_with_manual_ghg, enrich_ar6_with_hodnebrog
from flow_matching import match_flows

# for dummy database
from dummy import make_dummy_database




In [2]:
df = make_dummy_database()

biosphere3 already exists in current project.

Total biosphere flows extracted: 4709

Random 10 flows of biosphere3 with temporal data:
         date  amount  activity                   flow_name  \
0  2020-01-01       3         1                   Cobalt II   
1  2021-01-01       5         1                   Cobalt II   
2  2022-01-01       4         1                   Cobalt II   
3  2020-01-01       0         1  Methane, monochloro-, R-40   
4  2021-01-01       3         1  Methane, monochloro-, R-40   
5  2022-01-01       4         1  Methane, monochloro-, R-40   
6  2020-01-01       1         1                  Mesotrione   
7  2021-01-01       3         1                  Mesotrione   
8  2022-01-01       5         1                  Mesotrione   
9  2020-01-01       2         1                      Phenol   
10 2021-01-01       1         1                      Phenol   
11 2022-01-01       4         1                      Phenol   
12 2020-01-01       3         1              

In [4]:
# Step 1: Fetch IPCC AR6 GHG data from GitHub

df_ar6, ar6_cols = fetch_ar6_ghg_data()

Total rows in CSV: 249


In [5]:
# Step 2: Add manual entries for CO2, CH4, N2O with precise values

df_ar6 = enrich_ar6_with_manual_ghg(df_ar6, ar6_cols)

  Updated 'Carbon dioxide' -> CAS 124-38-9, Radiative_Efficiency 1.33e-05, Molar_Mass 0.04401
  Updated 'Methane' -> CAS 74-82-8, Radiative_Efficiency 0.000388, Molar_Mass 0.01604
  Updated 'Nitrous oxide' -> CAS 10024-97-2, Radiative_Efficiency 0.0032, Molar_Mass 0.04401


In [6]:
# Step 3: Fetch and enrich with Hodnebrog et al. (2020) data

df_hodne, hodne_cols = fetch_hodnebrog_data()

# Enrich AR6 data with Hodnebrog values
df_ar6, update_counts = enrich_ar6_with_hodnebrog(df_ar6, df_hodne, ar6_cols, hodne_cols)

Updated radiative efficiency from Hodnebrog for 246 rows.
Updated molar mass from Hodnebrog for 246 rows.

Remaining entities with no molar mass: 0


In [7]:
# Step 4: Filter for air compartment, making sure to exclude natural resource, soil, and water

df_air = apply_filter(df, CompartmentFilter("air"))
df_air = apply_filter(df_air, ExcludeCompartmentFilter(["natural resource", "soil", "water"]))
print(f"Total flows in 'air' compartment: {len(df_air)}")

Total flows in 'air' compartment: 1867


In [8]:
# Filter for carbon dioxide (CAS: 124-38-9) and methane (CAS: 74-82-8) using SpecificCASFilter
from filters import SpecificCASFilter

df_air_reset = df_air.reset_index().rename(columns={'index': 'orig_index'})

cas_targets = ["124-38-9", "74-82-8"]
df_co2_ch4 = apply_filter(df_air_reset, SpecificCASFilter(cas_targets))
matched_idx = df_co2_ch4['orig_index']

df_nonco2_nonch4 = df_air_reset[~df_air_reset['orig_index'].isin(matched_idx)] \
    .drop(columns=['orig_index']).reset_index(drop=True)

df_co2 = apply_filter(df_air_reset, SpecificCASFilter(["124-38-9"])) \
    .drop(columns=['orig_index']).reset_index(drop=True)
df_ch4 = apply_filter(df_air_reset, SpecificCASFilter(["74-82-8"])) \
    .drop(columns=['orig_index']).reset_index(drop=True)

print(f"Flows with CO2 (by CAS 124-38-9): {len(df_co2)}")
print(f"Flows with CH4 (by CAS 74-82-8): {len(df_ch4)}")
print(f"Flows excluding CO2 or CH4: {len(df_nonco2_nonch4)}")

Flows with CO2 (by CAS 124-38-9): 15
Flows with CH4 (by CAS 74-82-8): 15
Flows excluding CO2 or CH4: 1837


In [9]:
# Split non-CO2/non-CH4 flows by CAS presence
df_nonco2_nonch4_with_cas = apply_filter(df_nonco2_nonch4, CASNumberFilter(cas=True))
df_nonco2_nonch4_without_cas = apply_filter(df_nonco2_nonch4, CASNumberFilter(cas=False))

print(f"Non-CO2/non-CH4 flows WITH CAS: {len(df_nonco2_nonch4_with_cas)}")
print(f"Non-CO2/non-CH4 flows WITHOUT CAS: {len(df_nonco2_nonch4_without_cas)}")
print(f"Total non-CO2/non-CH4 flows: {len(df_nonco2_nonch4)}")

Non-CO2/non-CH4 flows WITH CAS: 1704
Non-CO2/non-CH4 flows WITHOUT CAS: 133
Total non-CO2/non-CH4 flows: 1837


In [21]:
print(df_nonco2_nonch4_without_cas.to_string())

                                              name                                     categories            unit      type location    database                                  code reference_product comment   CAS formula
0              Actinides, radioactive, unspecified                                         (air,)  kilo Becquerel  emission     None  biosphere3  d3805249-4c8b-48fb-8843-a7765b8d1322              None    None  None    None
1               Aerosols, radioactive, unspecified                                         (air,)  kilo Becquerel  emission     None  biosphere3  cec0687d-7d44-4e80-ae2f-428c2664d66d              None    None  None    None
2                           Aldehydes, unspecified                                         (air,)        kilogram  emission     None  biosphere3  06f0a67c-ec68-435d-8c32-0decd79a4a1a              None    None  None    None
3                           Alkylbenzene (c10-c15)                                         (air,)        kil

In [10]:
import importlib
import nmvoc_bp
importlib.reload(nmvoc_bp)

df_nonco2_nonch4_with_cas['CAS'] = df_nonco2_nonch4_with_cas['CAS'].apply(normalize_cas)

df_nonco2_nonch4_classified = nmvoc_bp.classify_nmvoc_bp(
    df_nonco2_nonch4_with_cas,  # Process all
    cache_db="nmvoc_cache.sqlite",
    cas_col="CAS",
    name_col="name",
    threshold_c=250.0,
    allow_estimates=False,  # Set to True if you want JOBACK estimates
    progress_every=50
)

# Split into three lists based on classification
df_nmvoc = df_nonco2_nonch4_classified[df_nonco2_nonch4_classified["nmvoc_status"] == "NMVOC"]
df_not_nmvoc = df_nonco2_nonch4_classified[df_nonco2_nonch4_classified["nmvoc_status"] == "NOT_NMVOC"]
df_unknown = df_nonco2_nonch4_classified[df_nonco2_nonch4_classified["nmvoc_status"] == "UNKNOWN"]

print(f"\nClassification results for non-CO2/non-CH4 flows:")
print(f"  NMVOC: {len(df_nmvoc)}")
print(f"  NOT_NMVOC: {len(df_not_nmvoc)}")
print(f"  UNKNOWN: {len(df_unknown)}")
print(f"  Total: {len(df_nonco2_nonch4_classified)}")


Classification results for non-CO2/non-CH4 flows:
  NMVOC: 682
  NOT_NMVOC: 608
  UNKNOWN: 414
  Total: 1704


In [13]:
# Find CAS column in biosphere data
flow_cas_col = None
for col in df.columns:
    if 'cas' in col.lower():
        flow_cas_col = col
        break

# Match NOT_NMVOC and UNKNOWN flows against AR6 table
# All matches (from both sets) -> df_other
# Non-matching from NOT_NMVOC -> df_not_other
# Non-matching from UNKNOWN -> overwrite df_unknown

df_not_other, df_other_not = None, None  # placeholders for clarity

# Match NOT_NMVOC
matched_not, not_matched_not = match_flows(
    df_not_nmvoc,
    df_ar6,
    flow_cas_col=flow_cas_col,
    ar6_cas_col=ar6_cols['cas'],
    ar6_cols={
        'lifetime': ar6_cols['lifetime'],
        'rad_eff': ar6_cols['rad_eff'],
        'molar_mass': ar6_cols['molar_mass']
    }
)

# Match UNKNOWN
matched_unknown, not_matched_unknown = match_flows(
    df_unknown,
    df_ar6,
    flow_cas_col=flow_cas_col,
    ar6_cas_col=ar6_cols['cas'],
    ar6_cols={
        'lifetime': ar6_cols['lifetime'],
        'rad_eff': ar6_cols['rad_eff'],
        'molar_mass': ar6_cols['molar_mass']
    }
)

# Combine matches from both into df_other
df_other = pd.concat([matched_not, matched_unknown], ignore_index=True)

# Store non-matching sets
df_not_other = not_matched_not
# Overwrite df_unknown with still-unmatched UNKNOWN rows
df_unknown = not_matched_unknown.reset_index(drop=True)

print(f"Matched from NOT_NMVOC: {len(matched_not)}")
print(f"Matched from UNKNOWN: {len(matched_unknown)}")
print(f"Total matched: {len(df_other)}")
print(f"Remaining NOT_NMVOC: {len(df_not_other)}")
print(f"Remaining UNKNOWN: {len(df_unknown)}")

Matched from NOT_NMVOC: 15
Matched from UNKNOWN: 0
Total matched: 15
Remaining NOT_NMVOC: 593
Remaining UNKNOWN: 414


In [23]:
print("Overview for characterization")
print(f"CO2 flows: {len(df_co2)}")
print(f"CH4 flows: {len(df_ch4)}")
print(f"NMVOC flows: {len(df_nmvoc)}")
print(f"Other GHGs flows: {len(df_other)}")
print(f"Flows where information for classification is insufficient: {len(df_unknown)}")

Overview for characterization
CO2 flows: 15
CH4 flows: 15
NMVOC flows: 682
Other GHGs flows: 15
Flows where information for classification is insufficient: 414


In [20]:
print(df_unknown.to_string())

                                      name                                     categories            unit      type location    database                                  code reference_product comment          CAS          formula   bp_c  is_organic nmvoc_status                           reason  molar_mass                                                                      source
0                              Acetamiprid                                         (air,)        kilogram  emission     None  biosphere3  01e5e64c-19ef-59c7-b350-7c46e691205a              None    None  135410-20-7             None    NaN       False      UNKNOWN              Formula unavailable         NaN  BP:[no Tb methods available] | Formula:[formula lookup failed: ValueError]
1                       Ammonium carbonate                                         (air,)        kilogram  emission     None  biosphere3  c36f35d1-e815-4c85-8686-1a2d21264a2c              None    None     506-87-6          CH8N2O3  

In [24]:
import requests
import math

def pubchem_fetch(cas: str, timeout: float = 8.0):
    """
    Fetch formula, molar mass, and boiling point (if present) from PubChem.
    Returns (formula, molar_mass_g_mol, bp_c) where any may be None.
    """
    # Fetch full record to mine Experimental Properties
    url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/{cas}/JSON"
    try:
        r = requests.get(url, timeout=timeout)
        r.raise_for_status()
        data = r.json()
    except Exception:
        return None, None, None

    # Helpers
    def find_sections(sections, target):
        for s in sections:
            if s.get("TOCHeading") == target:
                yield s
            for sub in find_sections(s.get("Section", []) or [], target):
                yield sub

    def first_boil_c(sections):
        for sec in find_sections(sections, "Boiling Point"):
            infos = sec.get("Information", []) or []
            for info in infos:
                vals = info.get("Value", {}).get("StringWithMarkup", []) or []
                for v in vals:
                    txt = v.get("String", "")
                    # naive parse: numbers before "°C"
                    if "°C" in txt:
                        try:
                            num = float(txt.split("°C")[0].strip().replace(",", ""))
                            return num
                        except Exception:
                            pass
                    # sometimes just number
                    try:
                        num = float(txt.strip().replace(",", ""))
                        return num
                    except Exception:
                        pass
        return None

    record = data.get("PC_Compounds", [{}])[0]
    props = record.get("props", []) or []
    atoms = record.get("atoms", {})
    elements = atoms.get("element", []) or []

    # Formula and molar mass from props
    formula = None
    mw = None
    for p in props:
        urn = p.get("urn", {})
        name = urn.get("name")
        label = urn.get("label")
        if name == "Molecular Formula" and not formula:
            sval = p.get("value", {}).get("sval")
            if sval:
                formula = sval.strip()
        if name == "Molecular Weight" and mw is None:
            fval = p.get("value", {}).get("fval")
            if fval is not None:
                mw = float(fval)

    # Boiling point from experimental properties
    sections = record.get("section", []) or []
    bp_c = first_boil_c(sections)

    return formula, mw, bp_c

def has_carbon(formula: str) -> bool:
    return bool(formula) and ("C" in formula) and not formula.strip().startswith("Cl")

def reclassify_online(df_unknown: pd.DataFrame, threshold_c: float = 250.0):
    df_u = df_unknown.copy()
    results = []
    for _, row in df_u.iterrows():
        cas = str(row["CAS"]).strip()
        formula, mw, bp_c = pubchem_fetch(cas)
        is_org = has_carbon(formula)
        if bp_c is not None:
            if bp_c >= threshold_c:
                status, reason = "NOT_NMVOC", f"BP {bp_c:.1f}°C >= {threshold_c}°C (online)"
            elif formula and is_org:
                status, reason = "NMVOC", f"Organic + BP {bp_c:.1f}°C < {threshold_c}°C (online)"
            else:
                status, reason = "UNKNOWN", "BP < threshold but formula missing/unclear (online)"
        else:
            if formula and not is_org:
                status, reason = "NOT_NMVOC", "Not organic (online)"
            elif formula and is_org:
                status, reason = "UNKNOWN", "Organic but BP unavailable (online)"
            else:
                status, reason = "UNKNOWN", "Formula & BP unavailable (online)"
        results.append((formula, mw, bp_c, is_org, status, reason))
    cols = ["formula_online", "molar_mass_online", "bp_c_online", "is_organic_online", "nmvoc_status_online", "reason_online"]
    df_u[cols] = results
    return df_u

# Run online enrichment only for UNKNOWN rows from Pass A
df_unknown_online = reclassify_online(df_unknown, threshold_c=250.0)

# Merge back: prefer online status if it resolves NMVOC/NOT_NMVOC
df_nonco2_nonch4_classified_online = df_nonco2_nonch4_classified.copy()
for idx, row in df_unknown_online.iterrows():
    if row["nmvoc_status_online"] in ("NMVOC", "NOT_NMVOC"):
        df_nonco2_nonch4_classified_online.loc[row.name, "nmvoc_status"] = row["nmvoc_status_online"]
        df_nonco2_nonch4_classified_online.loc[row.name, "reason"] = row["reason_online"]
        if pd.notna(row["bp_c_online"]):
            df_nonco2_nonch4_classified_online.loc[row.name, "bp_c"] = row["bp_c_online"]
        if pd.notna(row["formula_online"]):
            df_nonco2_nonch4_classified_online.loc[row.name, "formula"] = row["formula_online"]
        df_nonco2_nonch4_classified_online.loc[row.name, "is_organic"] = row["is_organic_online"]

# Summary
df_nonco2_nonch4_nmvoc_online = df_nonco2_nonch4_classified_online[df_nonco2_nonch4_classified_online["nmvoc_status"] == "NMVOC"]
df_nonco2_nonch4_not_nmvoc_online = df_nonco2_nonch4_classified_online[df_nonco2_nonch4_classified_online["nmvoc_status"] == "NOT_NMVOC"]
df_nonco2_nonch4_unknown_remaining = df_nonco2_nonch4_classified_online[df_nonco2_nonch4_classified_online["nmvoc_status"] == "UNKNOWN"]

print("\nOnline enrichment results (Pass B):")
print(f"  NMVOC (after online): {len(df_nonco2_nonch4_nmvoc_online)}")
print(f"  NOT_NMVOC (after online): {len(df_nonco2_nonch4_not_nmvoc_online)}")
print(f"  UNKNOWN (remaining): {len(df_nonco2_nonch4_unknown_remaining)}")

# Optional: inspect changes
print("\nSample resolved online:")
print(df_nonco2_nonch4_classified_online[df_nonco2_nonch4_classified_online.index.isin(df_nonco2_nonch4_unknown.index)].head(15)[["name","CAS","nmvoc_status","reason","bp_c","formula"]].to_string())


Online enrichment results (Pass B):
  NMVOC (after online): 682
  NOT_NMVOC (after online): 608
  UNKNOWN (remaining): 414

Sample resolved online:


NameError: name 'df_nonco2_nonch4_unknown' is not defined

In [ ]:
print(df_nonco2_nonch4_unknown.to_string())

                                       name                                     categories            unit      type location    database                                  code reference_product comment          CAS          formula   bp_c  is_organic nmvoc_status                           reason  molar_mass                                                                      source
3                               Acetamiprid                                         (air,)        kilogram  emission     None  biosphere3  01e5e64c-19ef-59c7-b350-7c46e691205a              None    None  135410-20-7             None    NaN       False      UNKNOWN              Formula unavailable         NaN  BP:[no Tb methods available] | Formula:[formula lookup failed: ValueError]
15                       Ammonium carbonate                                         (air,)        kilogram  emission     None  biosphere3  c36f35d1-e815-4c85-8686-1a2d21264a2c              None    None     506-87-6          CH8N2O

In [ ]:
# Check the quality of CAS numbers before classification
print("Sample CAS numbers from df_nonco2_with_cas:")
print(df_nonco2_with_cas[['name', 'CAS']].head(20).to_string())
print(f"\nUnique CAS numbers to process: {df_nonco2_with_cas['CAS'].nunique()}")
print(f"\nCAS number statistics:")
print(f"  Contains 'None': {df_nonco2_with_cas['CAS'].astype(str).str.lower().str.contains('none').sum()}")
print(f"  Contains 'nan': {df_nonco2_with_cas['CAS'].astype(str).str.lower().str.contains('nan').sum()}")
print(f"  Empty strings: {(df_nonco2_with_cas['CAS'].astype(str).str.strip() == '').sum()}")
print(f"  Valid format (contains '-'): {df_nonco2_with_cas['CAS'].astype(str).str.contains('-').sum()}")

Sample CAS numbers from df_nonco2_with_cas:
                       name          CAS
0              Acenaphthene      83-32-9
1            Acenaphthylene     208-96-8
2              Acetaldehyde      75-07-0
3               Acetamiprid  135410-20-7
4               Acetic acid      64-19-7
5   Acetic acid, trifluoro-      76-05-1
6                   Acetone      67-64-1
7              Acetonitrile      75-05-8
8                  Acrolein     107-02-8
9              Acrylic acid      79-10-7
10            Acrylonitrile     107-13-1
11       Alpha-cypermethrin   52315-07-8
12            Aluminium III   22537-23-1
13            Americium-241   14596-10-2
14                  Ammonia    7664-41-7
15       Ammonium carbonate     506-87-6
16                  Aniline      62-53-3
17               Anthracene     120-12-7
18         Anthranilic acid     118-92-3
19             Antimony-124   14683-10-4

Unique CAS numbers to process: 450

CAS number statistics:
  Contains 'None': 0
  Contains 'na

In [ ]:
# Debug: Check the classified results
print("Sample of classified data:")
print(df_nonco2_classified[['name', 'CAS', 'formula', 'bp_c', 'is_organic', 'nmvoc_status', 'reason', 'source']].head(20).to_string())

print(f"\n\nFormula value counts:")
print(f"  Non-null formulas: {df_nonco2_classified['formula'].notna().sum()}")
print(f"  Null formulas: {df_nonco2_classified['formula'].isna().sum()}")

print(f"\nOrganic status:")
print(f"  Organic (has carbon): {df_nonco2_classified['is_organic'].sum()}")
print(f"  Not organic: {(~df_nonco2_classified['is_organic']).sum()}")

print(f"\nBoiling point data:")
print(f"  Non-null BP: {df_nonco2_classified['bp_c'].notna().sum()}")
print(f"  Null BP: {df_nonco2_classified['bp_c'].isna().sum()}")

Sample of classified data:
                       name          CAS       formula    bp_c  is_organic nmvoc_status                              reason                                                                      source
0              Acenaphthene      83-32-9        C12H10  277.50        True    NOT_NMVOC              BP 277.50°C >= 250.0°C                                              BP:CRC_ORG | Formula:chemicals
1            Acenaphthylene     208-96-8         C12H8  280.00        True    NOT_NMVOC              BP 280.00°C >= 250.0°C                                              BP:CRC_ORG | Formula:chemicals
2              Acetaldehyde      75-07-0         C2H4O   20.80        True        NMVOC      Organic + BP 20.80°C < 250.0°C                                              BP:CRC_ORG | Formula:chemicals
3               Acetamiprid  135410-20-7          None     NaN       False      UNKNOWN                 Formula unavailable  BP:[no Tb methods available] | Formula:[formula 

In [ ]:
# Step 5: Split air flows by CAS presence

# Find CAS column in biosphere data
flow_cas_col = None
for col in df.columns:
    if 'cas' in col.lower():
        flow_cas_col = col
        break

# Apply filters
df_air_with_cas = apply_filter(df_air, CASNumberFilter(cas=True))
df_air_with_cas[flow_cas_col] = df_air_with_cas[flow_cas_col].apply(normalize_cas)

df_air_without_cas = apply_filter(df_air, CASNumberFilter(cas=False))

print(f"Air flows WITH CAS number: {len(df_air_with_cas)}")
print(f"Air flows WITHOUT CAS number: {len(df_air_without_cas)}")

Air flows WITH CAS number: 1742
Air flows WITHOUT CAS number: 135



Flows with CAS that MATCH CSV: 229
Flows with CAS that DON'T MATCH CSV: 1513

Matching flows (first 10):
                                         name categories      unit      type location    database                                  code reference_product comment         CAS formula  Lifetime [years]  Radiative_Efficiency [W m-2 ppb-1]  Molar_Mass [kg mol-1]
0                                Bromopropane     (air,)  kilogram  emission     None  biosphere3  f2e4394f-61bb-5493-983d-d3d5b7b96a41              None    None    106-94-5    None             0.041                            0.002340                0.12300
1                                      Butane     (air,)  kilogram  emission     None  biosphere3  982b0510-96ac-4bcb-a758-e98006b95f4d              None    None    106-97-8    None             0.019                            0.000290                0.05814
2                      Carbon dioxide, fossil     (air,)  kilogram  emission     None  biosphere3  349b29d1-3e58-4c66

In [ ]:
import importlib
import nmvoc_bp
importlib.reload(nmvoc_bp)

df_classified = nmvoc_bp.classify_nmvoc_bp(
    df_air_not_matched.head(30),
    cache_db="nmvoc_cache.sqlite",
    cas_col="CAS",
    name_col="name",
    threshold_c=250.0,
    allow_estimates=False,
    progress_every=50
)

df_nmvoc = df_classified[df_classified["nmvoc_status"] == "NMVOC"]
df_unknown = df_classified[df_classified["nmvoc_status"] == "UNKNOWN"]
df_no_nmvoc = df_classified[df_classified["nmvoc_status"] == "NOT_NMVOC"]

df_classified_ipcc = nmvoc_bp.classify_nmvoc_bp(
    df_air_matched,
    cache_db="nmvoc_cache.sqlite",
    cas_col="CAS",
    name_col="name",
    threshold_c=250.0,
    allow_estimates=False,
    progress_every=50
)

df_nmvoc_ipcc = df_classified_ipcc[df_classified_ipcc["nmvoc_status"] == "NMVOC"]

print('NMVOC list')
print(df_nmvoc.to_string())
#print('unknown')
#print(df_unknown.to_string())
#print('no nmvoc')
#print(df_no_nmvoc.to_string())

print('NMVOC ipcc list')
print(df_nmvoc_ipcc.to_string())

NMVOC list
                       name categories      unit      type location    database                                  code reference_product comment        CAS  formula    bp_c  is_organic nmvoc_status                           reason  molar_mass                          source
0       1,2-Dichlorobenzene     (air,)  kilogram  emission     None  biosphere3  b1c36287-329c-49f0-93c2-68246d34007c              None    None    95-50-1  C6H4Cl2  180.20        True        NMVOC  Organic + BP 180.20°C < 250.0°C   147.00196  BP:CRC_ORG | Formula:chemicals
1            1,4-Butanediol     (air,)  kilogram  emission     None  biosphere3  09db39be-d9a6-4fc3-8d25-1f80b23e9131              None    None   110-63-4  C4H10O2  229.50        True        NMVOC  Organic + BP 229.50°C < 250.0°C    90.12100  BP:CRC_ORG | Formula:chemicals
2                1-Pentanol     (air,)  kilogram  emission     None  biosphere3  048baf1e-6cdc-44a5-92e2-32d15ff54885              None    None    71-41-0   C5H12O  13